In [2]:
import torch 
import torchvision
import torchvision.transforms as tr 
from torch.utils.data import DataLoader, Dataset
import numpy as np 

## 1. 파이토치 제공 데이터 사용

논문에서 테스트용으로 많이 사용 (검증된 데이터이다)

In [4]:
#a. 데이터 전처리 설정 
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])

#compose 내에서 순서대로 전처리 
# 8x8로 크기 변환, 텐서 데이터로 변환 

In [5]:
#b. 데이터 다운받기 

# transform  데이터 전처리를 의미함 pad, grayscale 등등 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transf)


170499072it [02:01, 1402834.48it/s]                               


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:


trainset[0][0].size()
#튜플형태 이미지/데이터 
# 3channel, 8*8

torch.Size([3, 8, 8])

In [10]:
#c. 데이터 셋을 받아 우리가 원하는 배치 형태로 만들기 
#num_workers는 서브 프로세서를 몇개를 쓰는가 에러나면 0으로 설정하면 됨.
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

In [14]:
len(trainloader)

# 5만개 데이터, 배치 사이즈 = 50, 따라서 mini batch의 개수는 1000개

1000

In [15]:
# 한 묶음에 대해 불러오기 
dataiter = iter(trainloader)
images, labels = dataiter.next()
images.size()
# 배치 사이즈, 채널수, 8*8(image size)  

torch.Size([50, 3, 8, 8])

## 2. 같은 클래스 별 폴더 이미지 데이터 사용 

In [ ]:
#./class/tiger 
# ./class/lion

#전처리 작업 정의 
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
#레이블이 자동으로 매겨짐 + 데이터 전처리
trainset = torchvision.datasets.ImageFolder(root='./class', transform = transf)
#실제 활용을 위해 batchsize 만들음
trainloader = DataLoader(trainset, batch_size=1, shuffle=False, num_workers=2)
print(len(trainloader))

## 3. 개인 데이터 사용 + 전처리 
### type A 

In [16]:
#이미지 개수, 채널 수, 이미지 너비, 높이 
# 20개 32*32, 3channel 
train_images = np.random.randint(256, size=(20,32,32,3))
train_labels = np.random.randint(2, size=(20,1))

#전처리 코드 추가 진행해야함 

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [17]:
# data set을 상속받을 class 생성 

class TensorData(Dataset):
    #외부에서 데이터가 들어오는 구조 
        #폴더정리를 못하는 경우 
            # 1. 다른 작업과 공용으로 사용하는 경우 (거기서도 작업이 이루어져야하기 때문에 디렉을 바꿀 수 없음 )
            # 2. 폴더가 아닌 SQL같은 곳에서 넘어오는 경우 
        #파이토치 같은 경우 제공하는 전처리는 부족함 
    def __init__(self, x_data, y_data):
        #텐서 변환 
        self.x_data = torch.FloatTensor(x_data)
        #이미지 개수, 채널 수, 이미지 너비, 높이로 순서변환 
        self.x_data = self.x_data.permute(0,3,1,2) 
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]
    
    #데이터 외부로 내보냄 
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len 

In [18]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [19]:
#데이터가 이미지개수, 채널수, 크기로 알맞게 변했는지 확인 
dataiter = iter(train_loader)
images, labels = dataiter.next()
images.size()

torch.Size([10, 3, 32, 32])

## 3. 개인 데이터 사용 + 전처리 
### type B
우리가 가지고 있는 데이터를 파이토치 TRANSFORM 이용하여 전처리 

In [20]:
# data set을 상속받을 class 생성 

class MyDataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform 
        self.len = len(y_data)
        
    
    #데이터 외부로 내보냄 
    def __getitem__(self, index):
        sample = self.transform(index), self.y_data[index]
        
        #데이터 전처리를 하고 내보냄 
        #정의해놓은 트랜스폼이 없다면 (none)전처리를 하지 않고 내보냄 
        if self.transform :
            sample = self.transform(sample)
        
        return sample 
        
    def __len__(self):
        return self.len 
    
# 위 클래스는 type a와는 다르게 tensor 변환부가 없음 따라서 다른 클래스에서 정의함 
class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample 
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        return inputs, torch.LongTensor(labels)
    
class LinearTensor:
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias 
        
    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope*inputs + self.bias 
        return inputs, labels 
    



In [ ]:
trans = tr.Compose([ToTensor(), LinearTensor(2,5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
#batch 형태로 만들기 위해 dataloader 이용 
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)

만들어서 써야하는 이용 
데이터 타입(numpy)이 tr에 맞지 않기 때문에 스스로 정의하여 사용한다. 

In [ ]:
# data set을 상속받을 class 생성 

class MyDataset(Dataset):
    def __init__(self, x_data, y_data, transform=None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform 
        self.len = len(y_data)
        
    
    #데이터 외부로 내보냄 
    def __getitem__(self, index):
        sample = self.transform(index), self.y_data[index]
        
        #데이터 전처리를 하고 내보냄 
        #정의해놓은 트랜스폼이 없다면 (none)전처리를 하지 않고 내보냄 
        if self.transform :
            sample = self.transform(sample)
        
        return sample 
        
    def __len__(self):
        return self.len 
    
class MyTransform :
    def __call__(self, sample):
        inputs, labels = sample 
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        labels = torch.FloatTensor(labels)
        
        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize(0.5, 0.5, 0.5),(0.5,0.5,
        final_output=transf(inputs)
                             

In [ ]:
first_data = ds2[0]
features, labels = first_data
print(type(fatures), type(labels))